# Introduction to spark and EDA

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

## 1. Understanding Hadoop Distributed File System (HDFS)

Spark helps us for computation of big data sets. Hadoop has also this functionality with mapreduce but it is way faster spark. However, Spark does not have a file management system, whereas hadoop does. Here you can find a tutorial of how this HDFS works.

In [2]:
!hdfs

Usage: hdfs [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]

  OPTIONS is none or any of:

--buildpaths                       attempt to add class files from build tree
--config dir                       Hadoop config directory
--daemon (start|status|stop)       operate on a daemon
--debug                            turn on shell script debug mode
--help                             usage information
--hostnames list[,of,host,names]   hosts to use in worker mode
--hosts filename                   list of hosts to use in worker mode
--loglevel level                   set the log4j level for this command
--workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

cacheadmin           configure the HDFS cache
crypto               configure HDFS encryption zones
debug                run a Debug Admin to execute HDFS debug commands
dfsadmin             run a DFS admin client
dfsrouteradmin       manage Router-based federation
ec                   run a HD

In [3]:
!hdfs dfs -ls / # these are all the hadoop folders, we can see here the /ix folder

Found 14 items
drwxrwxrwt   - yarn   hadoop          0 2020-04-24 15:49 /app-logs
drwxr-xr-x   - hdfs   hdfs            0 2020-03-02 09:56 /apps
drwxr-xr-x   - yarn   hadoop          0 2020-02-04 08:46 /ats
drwxr-xr-x   - hdfs   hdfs            0 2020-02-04 08:47 /atsv2
drwxr-xr-x   - basil  hdfs            0 2020-04-08 12:26 /cs449
drwxr-xr-x   - hdfs   hdfs            0 2020-02-04 08:47 /hdp
drwxr-xr-x   - suresh hdfs            0 2020-02-11 16:43 /ix
drwxr-xr-x   - mapred hdfs            0 2020-02-04 08:47 /mapred
drwxrwxrwx   - mapred hadoop          0 2020-02-04 08:47 /mr-history
drwxr-xr-x   - hdfs   hdfs            0 2020-02-04 08:46 /services
drwxrwxrwx   - spark  hadoop          0 2020-04-24 15:56 /spark2-history
drwxrwxrwx   - hdfs   hdfs            0 2020-04-13 23:51 /tmp
drwxr-xr-x   - hdfs   hdfs            0 2020-04-24 15:49 /user
drwxr-xr-x   - hdfs   hdfs            0 2020-02-04 08:47 /warehouse


In [4]:
!hdfs dfs -ls /ix/ml-20m

Found 4 items
-rw-r--r--   3 suresh hdfs  745096004 2020-02-11 13:42 /ix/ml-20m/genome-scores.txt
-rw-r--r--   3 suresh hdfs      40652 2020-02-11 13:42 /ix/ml-20m/genome-tags.txt
-rw-r--r--   3 suresh hdfs    2538070 2020-02-11 13:42 /ix/ml-20m/movies.txt
-rw-r--r--   3 suresh hdfs 1493457002 2020-02-11 13:42 /ix/ml-20m/ratings.txt


In [5]:
!pwd # this is the linux space

/home/salvia/LAB3


In [6]:
!ls # again, this is whithin the linux space

ix-lab3-handout.pdf  lab3-recsys.ipynb	README.md
lab3-cluster.ipynb   most-rated.pickle	selected-movies.pickle
lab3-dimred.ipynb    rate-movies.py	snippets.ipynb


In [7]:
!hdfs dfs -mkdir /prova # We cannot manage this files... :( We are "clients"

mkdir: Permission denied: user=salvia, access=WRITE, inode="/":hdfs:hdfs:drwxr-xr-x


## 2. Exploratory data analysis

This is how the `genome-tags.txt` looks like:

In [8]:
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 5

{"tagId": 1124, "tag": "writing"}
{"tagId": 1125, "tag": "wuxia"}
{"tagId": 1126, "tag": "wwii"}
{"tagId": 1127, "tag": "zombie"}
{"tagId": 1128, "tag": "zombies"}


This is how `genome-scores.txt` looks like:

In [9]:
!hdfs dfs -cat /ix/ml-20m/genome-scores.txt | tail -n 1200 # this gives the relevance of each tag to each movie (percentage)

{"relevance": 0.14750000000000002, "tagId": 1057, "movieId": 131168}
{"relevance": 0.03475, "tagId": 1058, "movieId": 131168}
{"relevance": 0.044999999999999984, "tagId": 1059, "movieId": 131168}
{"relevance": 0.04049999999999998, "tagId": 1060, "movieId": 131168}
{"relevance": 0.025500000000000023, "tagId": 1061, "movieId": 131168}
{"relevance": 0.255, "tagId": 1062, "movieId": 131168}
{"relevance": 0.06, "tagId": 1063, "movieId": 131168}
{"relevance": 0.55375, "tagId": 1064, "movieId": 131168}
{"relevance": 0.27475000000000005, "tagId": 1065, "movieId": 131168}
{"relevance": 0.07474999999999998, "tagId": 1066, "movieId": 131168}
{"relevance": 0.00874999999999998, "tagId": 1067, "movieId": 131168}
{"relevance": 0.04199999999999998, "tagId": 1068, "movieId": 131168}
{"relevance": 0.004500000000000004, "tagId": 1069, "movieId": 131168}
{"relevance": 0.47875, "tagId": 1070, "movieId": 131168}
{"relevance": 0.15725, "tagId": 1071, "movieId": 131168}
{"relevance": 0.16899999999999998, "tag

This is how `ratings.txt` looks like:

In [10]:
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 5

{"movieId": 68954, "userId": 138493, "timestamp": 1258126920, "rating": 4.5}
{"movieId": 69526, "userId": 138493, "timestamp": 1259865108, "rating": 4.5}
{"movieId": 69644, "userId": 138493, "timestamp": 1260209457, "rating": 3.0}
{"movieId": 70286, "userId": 138493, "timestamp": 1258126944, "rating": 5.0}
{"movieId": 71619, "userId": 138493, "timestamp": 1255811136, "rating": 2.5}


This is how `movies.txt` looks like:

In [11]:
!hdfs dfs -cat /ix/ml-20m/movies.txt | tail -n 5

{"genres": ["Comedy"], "movieId": 131254, "title": "Kein Bund f\u00fcr's Leben (2007)"}
{"genres": ["Comedy"], "movieId": 131256, "title": "Feuer, Eis & Dosenbier (2002)"}
{"genres": ["Adventure"], "movieId": 131258, "title": "The Pirates (2014)"}
{"genres": ["(no genres listed)"], "movieId": 131260, "title": "Rentun Ruusu (2001)"}
{"genres": ["Adventure", "Fantasy", "Horror"], "movieId": 131262, "title": "Innocence (2014)"}


### START READING DATA:

Now that we have seen the data sets that we are working with lets read the data.

In [12]:
movies = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
tags = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
ratings = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)
tag_scores = sc.textFile("/ix/ml-20m/genome-scores.txt").map(json.loads)

IDM = dict(movies.map(itemgetter("movieId", "title")).collect())
IDT = dict(tags.map(itemgetter("tagId", "tag")).collect())

In [14]:
print("There are", len(IDM), "movies in the dataset.")
print("There are", len(IDT), "tags available.")
print("There are", ratings.count(), "ratings in our dataset.")
print("There are", tag_scores.count(), "scores in total for the tags. There are", len(IDT), "for each movie.")

There are 27278 movies in the dataset.
There are 1128 tags available.
There are 20000263 ratings in our dataset.
There are 11709768 scores in total for the tags. There are 1128 for each movie.


Now let us see how many movies have one tag or more:

In [66]:
print("If all movies had tags with relevance we would have a total of", 27278*1128, "scores")
print("Instead, we have", 11709768, ", which is a", (11709768/30769584)*100, "% of the total")

If all movies had tags with relevance we would have a total of 30769584 scores
Instead, we have 11709768 , which is a 38.05630911357137 % of the total


In [67]:
11709768/1128 # These are the number of evaluated movies

10381.0

In [37]:
ex = movies.filter(lambda x: x['genres'] == ["(no genres listed)"])

In [40]:
print("There are a total of", ex.count(), "movies without genres assigned.")

There are a total of 246 movies without genres assigned.


Now we would like to recuperate a list with all the genres used in our dataset.

1. We can save all the genres with a flatmap.
2. We can get rid of the ones that are repeated.

In [57]:
genres = movies.flatMap(itemgetter("genres")) # This is a list of lists. Each list containing a list of genres
genres = genres.distinct()

These are all the genres we are taking into account for each movie:

In [62]:
genres.take(100)

['Children',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Thriller',
 'Horror',
 'Sci-Fi',
 'IMAX',
 'Documentary',
 'Musical',
 'Western',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Mystery',
 'War',
 'Film-Noir',
 '(no genres listed)']

Which is the longest and shortest movie title?

In [63]:
movies.max(lambda x: len(x["title"]))

{'genres': ['Action', 'Fantasy', 'Sci-Fi'],
 'movieId': 27455,
 'title': 'Godzilla, Mothra, and King Ghidorah: Giant Monsters All-Out Attack (Gojira, Mosura, Kingu Gidorâ: Daikaijû sôkôgeki) (Godzilla, Mothra and King Ghidorah: Giant Monsters All-Out Attack) (2001)'}

In [64]:
movies.min(lambda x: len(x["title"]))

{'genres': ['Crime', 'Film-Noir', 'Thriller'],
 'movieId': 1260,
 'title': 'M (1931)'}